# RAG Chatbot - Jupyter Notebook Example

This notebook demonstrates how to use the RAG (Retrieval Augmented Generation) chatbot system from within a Jupyter environment.

## Features Covered:
- Basic chat functionality
- Conversation management
- Source document analysis
- Batch processing
- Data visualization
- Export capabilities

## Setup and Installation

In [ ]:
# Install required packages (uncomment if needed)
# !pip install requests pandas matplotlib seaborn plotly ipywidgets

# Import libraries
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import time
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Setup plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## RAG Client Configuration

In [ ]:
# Configuration
RAG_API_URL = "http://localhost:8000"  # Change this to your RAG service URL
# For AWS Lambda: "https://your-api-id.execute-api.region.amazonaws.com/dev"

TIMEOUT = 30
MAX_RETRIES = 3

# Initialize session
session = requests.Session()
session.headers.update({
    'Content-Type': 'application/json',
    'User-Agent': 'RAG-Jupyter-Client/1.0'
})

print(f"🚀 RAG Client configured for: {RAG_API_URL}")

## Helper Functions

In [ ]:
def check_health() -> bool:
    """Check if RAG service is healthy"""
    try:
        response = session.get(f"{RAG_API_URL}/health", timeout=TIMEOUT)
        return response.status_code == 200
    except:
        return False

def send_query(query: str, conversation_id: str = None, max_results: int = 3) -> Dict[str, Any]:
    """Send query to RAG service"""
    payload = {
        "query": query,
        "max_results": max_results
    }
    
    if conversation_id:
        payload["conversation_id"] = conversation_id
    
    try:
        response = session.post(
            f"{RAG_API_URL}/chat",
            json=payload,
            timeout=TIMEOUT
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            return {
                "error": f"HTTP {response.status_code}: {response.text}",
                "status": "error"
            }
    except Exception as e:
        return {
            "error": str(e),
            "status": "error"
        }

def format_response(response: Dict[str, Any]) -> str:
    """Format response for display"""
    if response.get("status") == "error":
        return f"❌ Error: {response.get('error', 'Unknown error')}"
    
    output = []
    output.append(f"🤖 **Answer:** {response.get('answer', 'No answer provided')}")
    
    sources = response.get("sources", [])
    if sources:
        output.append("\n📚 **Sources:**")
        for i, source in enumerate(sources[:3], 1):
            source_name = source.get("source", "Unknown")
            score = source.get("score", 0)
            page = source.get("page")
            page_info = f" (page {page})" if page else ""
            output.append(f"  {i}. {source_name}{page_info} - Relevance: {score:.2f}")
    
    processing_time = response.get("processing_time")
    if processing_time:
        cached_info = " (cached)" if response.get("cached") else ""
        output.append(f"\n⏱️ Processing time: {processing_time:.2f}s{cached_info}")
    
    return "\n".join(output)

# Test connection
if check_health():
    print("✅ RAG service is healthy")
else:
    print("❌ RAG service is not responding. Please check the URL and service status.")

## Basic Chat Functionality

In [ ]:
# Simple query example
query = "What is machine learning?"
response = send_query(query)

print(f"Query: {query}")
print("=" * 50)
print(format_response(response))

## Conversation Management

In [ ]:
# Start a conversation
conversation_id = None
conversation_history = []

queries = [
    "What is artificial intelligence?",
    "How does it relate to machine learning?",
    "Can you give me some examples?"
]

print("🗣️ **Conversation Example**\n")

for i, query in enumerate(queries, 1):
    print(f"**Turn {i}**")
    print(f"👤 Human: {query}")
    
    response = send_query(query, conversation_id)
    
    if response.get("status") != "error":
        conversation_id = response.get("conversation_id")
        answer = response.get("answer", "No answer")
        print(f"🤖 Assistant: {answer}")
        
        # Store conversation
        conversation_history.append({
            "turn": i,
            "query": query,
            "response": response,
            "timestamp": datetime.now()
        })
    else:
        print(f"❌ Error: {response.get('error')}")
    
    print("-" * 60)

print(f"\n📝 Conversation ID: {conversation_id}")
print(f"📊 Total turns: {len(conversation_history)}")

## Batch Query Processing

In [ ]:
# Batch processing example
batch_queries = [
    "What is deep learning?",
    "Explain neural networks",
    "What is natural language processing?",
    "How does computer vision work?",
    "What are the applications of AI in healthcare?"
]

batch_results = []
start_time = time.time()

print(f"🔄 Processing {len(batch_queries)} queries...\n")

for i, query in enumerate(batch_queries, 1):
    print(f"Processing {i}/{len(batch_queries)}: {query[:50]}...")
    
    query_start = time.time()
    response = send_query(query)
    query_time = time.time() - query_start
    
    result = {
        "query": query,
        "response": response,
        "query_time": query_time,
        "timestamp": datetime.now(),
        "success": response.get("status") != "error"
    }
    
    batch_results.append(result)
    
    # Small delay to avoid overwhelming the service
    time.sleep(0.5)

total_time = time.time() - start_time
successful_queries = sum(1 for r in batch_results if r["success"])

print(f"\n✅ Batch processing completed!")
print(f"📊 Success rate: {successful_queries}/{len(batch_queries)} ({successful_queries/len(batch_queries)*100:.1f}%)")
print(f"⏱️ Total time: {total_time:.2f}s")
print(f"⚡ Average time per query: {total_time/len(batch_queries):.2f}s")

## Data Analysis and Visualization

In [ ]:
# Create DataFrame from batch results
df_results = pd.DataFrame([
    {
        'query': r['query'],
        'query_length': len(r['query']),
        'processing_time': r['response'].get('processing_time', 0),
        'query_time': r['query_time'],
        'cached': r['response'].get('cached', False),
        'num_sources': len(r['response'].get('sources', [])),
        'success': r['success'],
        'answer_length': len(r['response'].get('answer', '')) if r['success'] else 0
    }
    for r in batch_results
])

print("📊 **Batch Results Summary:**")
display(df_results.describe())

In [ ]:
# Processing time analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('RAG System Performance Analysis', fontsize=16)

# Processing time distribution
axes[0, 0].hist(df_results['processing_time'], bins=10, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Processing Time Distribution')
axes[0, 0].set_xlabel('Processing Time (s)')
axes[0, 0].set_ylabel('Frequency')

# Query length vs processing time
axes[0, 1].scatter(df_results['query_length'], df_results['processing_time'], alpha=0.7)
axes[0, 1].set_title('Query Length vs Processing Time')
axes[0, 1].set_xlabel('Query Length (characters)')
axes[0, 1].set_ylabel('Processing Time (s)')

# Number of sources returned
axes[1, 0].bar(range(len(df_results)), df_results['num_sources'], alpha=0.7, color='lightgreen')
axes[1, 0].set_title('Number of Sources per Query')
axes[1, 0].set_xlabel('Query Index')
axes[1, 0].set_ylabel('Number of Sources')

# Answer length distribution
successful_results = df_results[df_results['success']]
axes[1, 1].hist(successful_results['answer_length'], bins=10, alpha=0.7, color='orange')
axes[1, 1].set_title('Answer Length Distribution')
axes[1, 1].set_xlabel('Answer Length (characters)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## Source Analysis

In [ ]:
# Analyze source documents
all_sources = []

for result in batch_results:
    if result['success']:
        sources = result['response'].get('sources', [])
        for source in sources:
            source_info = {
                'query': result['query'],
                'source_name': source.get('source', 'Unknown'),
                'score': source.get('score', 0),
                'page': source.get('page'),
                'text_length': len(source.get('text', ''))
            }
            all_sources.append(source_info)

if all_sources:
    df_sources = pd.DataFrame(all_sources)
    
    print(f"📚 **Source Documents Analysis** ({len(all_sources)} total sources)\n")
    
    # Most frequently cited sources
    source_counts = df_sources['source_name'].value_counts().head(10)
    print("**Most Frequently Cited Sources:**")
    for source, count in source_counts.items():
        print(f"  • {source}: {count} times")
    
    # Average relevance scores
    avg_scores = df_sources.groupby('source_name')['score'].agg(['mean', 'count']).round(3)
    avg_scores = avg_scores[avg_scores['count'] >= 2].sort_values('mean', ascending=False)
    
    print("\n**Average Relevance Scores (sources with 2+ citations):**")
    display(avg_scores.head(10))
    
    # Visualize source distribution
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    source_counts.head(8).plot(kind='bar')
    plt.title('Top Source Documents')
    plt.xlabel('Source Document')
    plt.ylabel('Citation Count')
    plt.xticks(rotation=45, ha='right')
    
    plt.subplot(1, 2, 2)
    plt.hist(df_sources['score'], bins=20, alpha=0.7, color='lightcoral')
    plt.title('Relevance Score Distribution')
    plt.xlabel('Relevance Score')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
else:
    print("No source information available from the queries.")

## Interactive Query Interface

In [ ]:
# Interactive widget-based interface (requires ipywidgets)
try:
    from ipywidgets import interact, widgets, VBox, HBox
    from IPython.display import display, clear_output, Markdown
    
    # Create widgets
    query_input = widgets.Text(
        placeholder='Enter your question here...',
        description='Query:',
        layout=widgets.Layout(width='500px')
    )
    
    max_results_slider = widgets.IntSlider(
        value=3,
        min=1,
        max=10,
        description='Max Results:'
    )
    
    send_button = widgets.Button(
        description='Send Query',
        button_style='primary',
        icon='paper-plane'
    )
    
    clear_button = widgets.Button(
        description='Clear Output',
        button_style='warning',
        icon='trash'
    )
    
    output_area = widgets.Output()
    
    # Widget interactions
    def send_query_widget(b):
        with output_area:
            query = query_input.value.strip()
            if not query:
                print("Please enter a query.")
                return
            
            print(f"🔍 Processing: {query}")
            response = send_query(query, max_results=max_results_slider.value)
            print("\n" + format_response(response))
            print("\n" + "="*60 + "\n")
    
    def clear_output_widget(b):
        output_area.clear_output()
    
    # Connect button events
    send_button.on_click(send_query_widget)
    clear_button.on_click(clear_output_widget)
    
    # Handle Enter key
    def handle_enter(sender):
        send_query_widget(None)
    
    query_input.on_submit(handle_enter)
    
    # Display interface
    print("🎛️ **Interactive Query Interface**")
    
    interface = VBox([
        HBox([query_input, max_results_slider]),
        HBox([send_button, clear_button]),
        output_area
    ])
    
    display(interface)
    
except ImportError:
    print("💡 Install ipywidgets for interactive interface: pip install ipywidgets")
    print("   Then restart the kernel and run this cell again.")

## Export and Reporting

In [ ]:
# Export results to various formats
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Export to JSON
export_data = {
    'timestamp': timestamp,
    'api_url': RAG_API_URL,
    'batch_results': [
        {
            'query': r['query'],
            'response': r['response'],
            'processing_time': r['query_time'],
            'timestamp': r['timestamp'].isoformat(),
            'success': r['success']
        }
        for r in batch_results
    ],
    'conversation_history': [
        {
            'turn': h['turn'],
            'query': h['query'],
            'response': h['response'],
            'timestamp': h['timestamp'].isoformat()
        }
        for h in conversation_history
    ],
    'summary': {
        'total_queries': len(batch_results),
        'successful_queries': successful_queries,
        'success_rate': successful_queries / len(batch_results) if batch_results else 0,
        'average_processing_time': df_results['processing_time'].mean() if not df_results.empty else 0,
        'total_sources_found': len(all_sources) if all_sources else 0
    }
}

json_filename = f"rag_session_{timestamp}.json"
with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False, default=str)

print(f"💾 Session data exported to: {json_filename}")

# Export DataFrame to CSV
if not df_results.empty:
    csv_filename = f"rag_results_{timestamp}.csv"
    df_results.to_csv(csv_filename, index=False)
    print(f"📊 Results DataFrame exported to: {csv_filename}")

# Generate summary report
print("\n" + "="*60)
print("📋 **SESSION SUMMARY REPORT**")
print("="*60)
print(f"🕐 Session time: {timestamp}")
print(f"🔗 API URL: {RAG_API_URL}")
print(f"📝 Total queries processed: {len(batch_results)}")
print(f"✅ Successful queries: {successful_queries}")
print(f"📊 Success rate: {successful_queries/len(batch_results)*100:.1f}%" if batch_results else "N/A")
if not df_results.empty:
    print(f"⏱️ Average processing time: {df_results['processing_time'].mean():.2f}s")
    print(f"📚 Total sources retrieved: {len(all_sources)}")
    print(f"🔍 Average sources per query: {df_results['num_sources'].mean():.1f}")
print(f"🗣️ Conversation turns: {len(conversation_history)}")
print("="*60)

## Advanced Features

In [ ]:
# Performance benchmarking
def benchmark_queries(queries: List[str], iterations: int = 3) -> Dict[str, Any]:
    """Benchmark query performance"""
    results = []
    
    for query in queries:
        query_times = []
        for i in range(iterations):
            start_time = time.time()
            response = send_query(query)
            end_time = time.time()
            
            if response.get('status') != 'error':
                query_times.append(end_time - start_time)
            
            time.sleep(0.1)  # Small delay between iterations
        
        if query_times:
            results.append({
                'query': query,
                'avg_time': sum(query_times) / len(query_times),
                'min_time': min(query_times),
                'max_time': max(query_times),
                'iterations': len(query_times)
            })
    
    return results

# Run benchmark
benchmark_queries_sample = [
    "What is AI?",
    "Explain machine learning algorithms",
    "How does natural language processing work in modern applications?"
]

print("🏃‍♂️ Running performance benchmark...")
benchmark_results = benchmark_queries(benchmark_queries_sample, iterations=3)

if benchmark_results:
    df_benchmark = pd.DataFrame(benchmark_results)
    
    print("\n⚡ **Performance Benchmark Results:**")
    display(df_benchmark)
    
    # Visualize benchmark results
    plt.figure(figsize=(12, 6))
    
    x_pos = range(len(df_benchmark))
    plt.bar(x_pos, df_benchmark['avg_time'], 
            yerr=[df_benchmark['avg_time'] - df_benchmark['min_time'],
                  df_benchmark['max_time'] - df_benchmark['avg_time']],
            capsize=5, alpha=0.7, color='lightblue')
    
    plt.xlabel('Query')
    plt.ylabel('Response Time (seconds)')
    plt.title('Query Performance Benchmark')
    plt.xticks(x_pos, [f"Q{i+1}" for i in range(len(df_benchmark))], rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Performance summary
    print(f"\n📊 **Benchmark Summary:**")
    print(f"  • Average response time: {df_benchmark['avg_time'].mean():.2f}s")
    print(f"  • Fastest query: {df_benchmark['min_time'].min():.2f}s")
    print(f"  • Slowest query: {df_benchmark['max_time'].max():.2f}s")
else:
    print("❌ Benchmark failed - no successful queries")

## Conclusion

This notebook demonstrated comprehensive usage of the RAG chatbot system including:

- ✅ Basic query functionality
- ✅ Conversation management
- ✅ Batch processing capabilities
- ✅ Performance analysis and visualization
- ✅ Source document analysis
- ✅ Interactive query interface
- ✅ Data export and reporting
- ✅ Performance benchmarking

The RAG system provides a powerful interface for accessing and querying your enterprise knowledge base with context-aware responses and source attribution.